In [ ]:
# 데이터 전처리용
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
!pip install konlpy
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
okt = Okt()

# 데이터 호출
import pandas as pd
import numpy as np

# 기계학습
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import os
import pickle
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')  # 경고메시지 숨기기

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import torch
import torch.nn as nn

from tensorflow import nn
from tensorflow import function, GradientTape, sqrt, abs, reduce_mean, ones_like, zeros_like, convert_to_tensor,float32
from tensorflow import data as tfdata
from tensorflow import config as tfconfig
from tensorflow.keras import Model, Sequential, Input
from tensorflow.keras.layers import GRU, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
!pip install transformers
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from IPython.display import Image
from tqdm import tqdm, trange

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 48.5 MB/s 
     |████████████████████████████████| 182 kB 68.3 MB/s 


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/플젝/보아즈감성분석/data.csv')

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/플젝/보아즈감성분석/okt.csv')

In [ ]:
df=df.drop('Unnamed: 0',axis=1)

In [ ]:
data.head()

,Unnamed: 0,date,close,diff,open,high,low,volume,updown,등락률,날짜,언론사,제목,링크
0,0,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,매경이코노미,‘오딘 개발사’ 품었다…카카오게임즈 주가 10% 급등,https://www.mk.co.kr/economy/view/2021/1034761
1,1,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,국민일보,"카카오게임즈, ‘오딘’ 개발사 품는다",http://news.kmib.co.kr/article/view.asp?arcid=...
2,2,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,연합뉴스,"카카오게임즈, '오딘' 개발사 라이온하트 최대주주 된다(종합)",http://yna.kr/AKR20211101072551002?did=1195m
3,3,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,조선비즈,"카카오게임즈, 2090억원 규모의 유상증자 결정",https://biz.chosun.com/stock/stock_general/202...
4,4,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,한국경제TV,"""게임주 극성수기 온다""…기관은 카카오게임즈 `찜` [박해린의 뉴스&마켓]",https://www.wowtv.co.kr/NewsCenter/News/Read?a...


In [ ]:
df=data[['등락률','제목','updown']]

In [ ]:
df

,제목
0,오딘 개발 품다 카카오 게임 주가 급등
1,카카오 게임 오딘 개발 품다
2,카카오 게임 오딘 개발 라이온하트 최대 주주 되다 종합
3,카카오 게임 규모 유상증자 결정
4,게임 주 극성 수기 오다 기관 카카오 게임 찜 박해 린 뉴스 마켓
...,...
54915,나흘 대장정 마치다 지스타 안전하다 게임 문화 즐기다
54916,지스타 폐막 총 관람객 잠정 집계
54917,지스타 성황리 폐막 나흘 방문
54918,지스타 게임 뭉치다 살 지스타 성숙하다 게임 축제 거듭


In [ ]:
import os
import torch
# gpu 설정
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"    # gpu 할당 in jupyter
os.environ["CUDA_VISIBLE_DEVICES"]= "4"

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC").to(device)

Downloading:   0%|          | 0.00/372 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/143k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/294k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/881 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406M [00:00<?, ?B/s]

In [ ]:
X=df['제목']

In [ ]:
tokenizer.tokenize(X[0])

['오', '##딘', '개발', '품', '##다', '카카오', '게임', '주가', '급', '##등']

In [ ]:
outputs_list = []
for text in tqdm(X) :
    inputs = tokenizer(text, return_tensors='pt').to(device)
    output = model(**inputs)
    output = output.logits.tolist()[0]
    outputs_list.append(output)
#출력값
outputs = torch.tensor(outputs_list) 

100%|██████████| 54920/54920 [1:43:24<00:00,  8.85it/s]


In [ ]:
outputs

tensor([[-3.6149, -2.5250,  6.4303],
        [-1.1920,  1.2916, -0.0092],
        [-2.2760, -2.1224,  4.5535],
        ...,
        [ 3.4101,  0.3673, -3.8511],
        [-2.4964,  5.8977, -2.7415],
        [-3.0083,  6.7556, -3.2511]])

In [ ]:
# 확률값으로 변경
predictions=nn.softmax(outputs, -1)
#출력값 확인
df_sc = pd.DataFrame(predictions.numpy())
df_sc.columns = ['부정','중립','긍정']
df_sc

,부정,중립,긍정
0,0.000043,0.000129,0.999828
1,0.061545,0.737591,0.200864
2,0.001079,0.001258,0.997663
3,0.000058,0.999817,0.000125
4,0.563029,0.397754,0.039216
...,...,...,...
54915,0.000309,0.999597,0.000094
54916,0.000302,0.999623,0.000075
54917,0.953832,0.045498,0.000670
54918,0.000226,0.999597,0.000177


In [ ]:
df=df.rename(columns={'제목':'토큰'})

In [ ]:
df2 = pd.concat([data[['updown','등락률','제목']],df, df_sc], axis=1)
df2

,updown,등락률,제목,토큰,부정,중립,긍정
0,-0.071672,0.0,‘오딘 개발사’ 품었다…카카오게임즈 주가 10% 급등,오딘 개발 품다 카카오 게임 주가 급등,0.000043,0.000129,0.999828
1,-0.071672,0.0,"카카오게임즈, ‘오딘’ 개발사 품는다",카카오 게임 오딘 개발 품다,0.061545,0.737591,0.200864
2,-0.071672,0.0,"카카오게임즈, '오딘' 개발사 라이온하트 최대주주 된다(종합)",카카오 게임 오딘 개발 라이온하트 최대 주주 되다 종합,0.001079,0.001258,0.997663
3,-0.071672,0.0,"카카오게임즈, 2090억원 규모의 유상증자 결정",카카오 게임 규모 유상증자 결정,0.000058,0.999817,0.000125
4,-0.071672,0.0,"""게임주 극성수기 온다""…기관은 카카오게임즈 `찜` [박해린의 뉴스&마켓]",게임 주 극성 수기 오다 기관 카카오 게임 찜 박해 린 뉴스 마켓,0.563029,0.397754,0.039216
...,...,...,...,...,...,...,...
54915,0.002232,1.0,나흘간의 대장정 마친 '지스타 2022'...18만명이 안전하게 '게임문화' 즐겼다,나흘 대장정 마치다 지스타 안전하다 게임 문화 즐기다,0.000309,0.999597,0.000094
54916,0.002232,1.0,지스타 2022 폐막! 4일간 총 관람객 18만 4천여명으로 잠정 집계,지스타 폐막 총 관람객 잠정 집계,0.000302,0.999623,0.000075
54917,0.002232,1.0,'지스타 2022' 성황리 폐막…나흘간 18만명 방문,지스타 성황리 폐막 나흘 방문,0.953832,0.045498,0.000670
54918,0.002232,1.0,"[지스타]""게임으로 뭉쳤다""...18살 '지스타' 성숙한 게임 축제로 거듭나",지스타 게임 뭉치다 살 지스타 성숙하다 게임 축제 거듭,0.000226,0.999597,0.000177


In [ ]:
df2.to_csv('/content/drive/MyDrive/Colab Notebooks/플젝/보아즈감성분석/finberttoken.csv', index=False)

## 여기부터 새로

In [ ]:
import pandas as pd
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/플젝/보아즈감성분석/finberttoken.csv')
df_sc=df2[['부정','중립','긍정']]

In [ ]:
df_sc['감성']=df_sc.idxmax(1)
df_sc['감성수치']=df_sc.max(1)

<ipython-input-3-38a8a6d2529d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sc['감성']=df_sc.idxmax(1)
<ipython-input-3-38a8a6d2529d>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sc['감성수치']=df_sc.max(1)
<ipython-input-3-38a8a6d2529d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sc['감성수치']=df_sc.max(1)


In [ ]:
df_sc['감성'].value_counts()

중립    31555
긍정    15939
부정     7426
Name: 감성, dtype: int64

In [ ]:
df=pd.concat([df_sc['감성'],df2[['토큰','제목']]],axis=1)

In [ ]:
df

,감성,토큰,제목
0,긍정,오딘 개발 품다 카카오 게임 주가 급등,‘오딘 개발사’ 품었다…카카오게임즈 주가 10% 급등
1,중립,카카오 게임 오딘 개발 품다,"카카오게임즈, ‘오딘’ 개발사 품는다"
2,긍정,카카오 게임 오딘 개발 라이온하트 최대 주주 되다 종합,"카카오게임즈, '오딘' 개발사 라이온하트 최대주주 된다(종합)"
3,중립,카카오 게임 규모 유상증자 결정,"카카오게임즈, 2090억원 규모의 유상증자 결정"
4,부정,게임 주 극성 수기 오다 기관 카카오 게임 찜 박해 린 뉴스 마켓,"""게임주 극성수기 온다""…기관은 카카오게임즈 `찜` [박해린의 뉴스&마켓]"
...,...,...,...
54915,중립,나흘 대장정 마치다 지스타 안전하다 게임 문화 즐기다,나흘간의 대장정 마친 '지스타 2022'...18만명이 안전하게 '게임문화' 즐겼다
54916,중립,지스타 폐막 총 관람객 잠정 집계,지스타 2022 폐막! 4일간 총 관람객 18만 4천여명으로 잠정 집계
54917,부정,지스타 성황리 폐막 나흘 방문,'지스타 2022' 성황리 폐막…나흘간 18만명 방문
54918,중립,지스타 게임 뭉치다 살 지스타 성숙하다 게임 축제 거듭,"[지스타]""게임으로 뭉쳤다""...18살 '지스타' 성숙한 게임 축제로 거듭나"


# 1번째 : finbert로 긍/부정 나누고 분류하기

In [ ]:

def encode_sentiments_values(df2):
    
    possible_sentiments = df2.감성.unique()
    sentiment_dict = {}
    
    for index, possible_sentiment in enumerate(possible_sentiments):
        sentiment_dict[possible_sentiment] = index
    
    # Encode all the sentiment values
    df['label'] = df2.감성.replace(sentiment_dict)
    
    return df, sentiment_dict
 
# Perform the encoding task on the data set
financial_data, sentiment_dict = encode_sentiments_values(df)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(financial_data['토큰'], financial_data['label'], test_size=0.3, random_state=777, stratify=financial_data['label']) 

In [ ]:
finbert_tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")

# Encode the Training data
encoded_data_train = finbert_tokenizer.batch_encode_plus(
    X_train.values.tolist(), 
    return_tensors='pt',
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=150 # the maximum lenght observed in the headlines
)

# Encode the Validation data
encoded_data_val = finbert_tokenizer.batch_encode_plus(
    X_val.values.tolist(), 
    return_tensors='pt',
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=150 # the maximum lenght observed in the headlines
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
sentiments_val = torch.tensor(y_test.values)

from torch.utils.data import  TensorDataset, DataLoader

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, sentiments_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
from torch.utils.data import Dataset, ConcatDataset, Sampler, RandomSampler, BatchSampler, SequentialSampler
batch_size = 100
from transformers import get_linear_schedule_with_warmup

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)


optimizer = torch.optim.Adam(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
epochs = 1

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in sentiment_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

seed_val = 2022

np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    torch.save(model.state_dict(), f'finetuned_finBERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

Epoch 1:   0%|          | 0/385 [00:00<?, ?it/s]